In [8]:
import Tkinter as tk
import tkFileDialog as tkfd
import tkMessageBox as tkmb
import os
import sys
import json
import itertools
from DINGO.utils import read_config
from DINGO.base import DINGO
from pprint import pprint

loremipsum = ('Lorem ipsum dolor sit amet, consectetur adipiscing elit, '
              'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. '
              'Ut enim ad minim veniam, quis nostrud exercitation ullamco '
              'laboris nisi ut aliquip ex ea commodo consequat. '
              'Duis aute irure dolor in reprehenderit in voluptate velit '
              'esse cillum dolore eu fugiat nulla pariatur. '
              'Excepteur sint occaecat cupidatat non proident, '
              'sunt in culpa qui officia deserunt mollit anim id est laborum.')

shortlorem = 'Lorem ipsum dolor sit amet,'

veryshortlorem = 'Lorem ipsum'

class Extra_entry(tk.Frame):
    def __init__(self, parent, **kwargs):
        tk.Frame.__init__(self, master=parent)
        self.create_widgets()
        self.grid_widgets()
        if kwargs is not None:
            for k, v in kwargs.iteritems():
                self.add_entry(k, v)
            
    def create_widgets(self):
        self.scrollbar = tk.Scrollbar(self)
        self.entries = tk.Listbox(self, yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.entries.yview)
            
    def grid_widgets(self):
        self.entries.grid(row=0, column=0)
        self.scrollbar.grid(row=0, column=1)
    
    @staticmethod
    def enum_reversed(alist):
        for idx in reversed(xrange(len(alist))):
            yield idx, alist[idx]
            
    def callback(self):
        for idx, pair in Extra_entry.enum_reversed(self.entries):
            var = pair[0]
            entry = pair[1]
            if var.get() == '':
                if idx != 0 and idx != len(self.entries)-1:
                    self.entries_listbox.delete(idx)
                    entry.destroy()
                    del self.entries[idx]
            else:
                self.add_entry()
        
    def add_entry(self):
        newvar = tk.StringVar(self)
        self.entries.append([newvar, 
                             tk.Entry(self,
                                 textvariable=newvar,
                                 validate='focusout',
                                 validatecommand=self.callback)])
        self.entries_listbox.insert('END', self.entries[-1][1])
root = tk.Tk()
Extra_entry(root)
root.mainloop()

In [1]:

            
class DINGOOptions(tk.Frame):
    def __init__(self, parent, config=None):
        tk.Frame.__init__(self, master=parent)
        self.create_widgets()
        self.grid_widgets()
    
    def create_widgets(self):
        self.option_frame = tk.Frame(master=self)
        self.option_scroll = tk.Scrollbar(master=self.option_frame, orient='VERTICAL')
        self.option_list = tk.Listbox(master=self.option_frame, yscrollcommand=self.option_scroll.set)
        self.option_scroll.config(command=self.option_list.yview)
        self.option_current = None
        self.entry_frame = tk.Frame(master=self)
        self.entry_scroll = tk.Scrollbar(master=self.entry_frame, orient='VERTICAL')
        self.entry_list = tk.Listbox(master=self.entry_frame, yscrollcommand=self.entry_scroll.set)
        self.entry_scroll.config(command=self.entry_list.yview)
        self.entry_current = None
        
    def grid_widgets(self):
        self.option_frame.grid(row=0, column=0)
        self.entry_frame.grid(row=0, column=1)
        
        self.option_list.grid(row=0, column=0)
        self.option_scroll.grid(row=0, column=1)
        self.entry_list.grid(row=0, column=0)
        self.entry_scroll.grid(row=0, column=1)
        
    def make_options_widget(self, k, v):
        print('MkOp: k: {}, v: {}'.format(k,v))
        self.option_list.insert('END', k)
                
            
    def make_input_widget(self, aninput):
        pass
    
    def poll(self):
        current_option = self.option_list.curselection()
        if current_option != self.option_current:
            self.view_option_entries(current_option)
            self.option_current = current_option
        self.after(250, self.poll)
        
    def view_option_entries(option):
        
        
class DINGOFile(tk.Frame):
    def __init__(self, parent, filename=None):
        tk.Frame.__init__(self, master=parent)
        self.create_widgets(parent, filename)
        self.grid_widgets()
        
    def create_widgets(self, parent, filename):
        if filename is None:
            default_filename = ''
        self.file_label = tk.Label(self, text=default_filename)
        self.new_button = tk.Button(self, text='New', command=lambda : self.new_config(parent))
        self.open_button = tk.Button(self, text='Open', command=lambda : self.ask_open_json(parent, filename))
        self.save_button = tk.Button(self, text='Save', command=lambda : self.save_json)
        
    def grid_widgets(self):    
        self.file_label.grid(row=0, column=0)
        self.new_button.grid(row=1, column=0)
        self.open_button.grid(row=1, column=1)
        self.save_button.grid(row=1, column=2)
        
    def new_config(self, parent):
        keys = []
        for tup in DINGO.default_expected_keys:
            if isinstance(tup[0],(unicode,str)):
                keys.append(tup[0])
            elif isinstance(tup[0],(tuple,list)):
                keys.append(tup[0][0])
        config = dict(zip(keys, itertools.repeat(None)))
        parent.update_config(config)
        
    def ask_open_json(self, parent, filename):
        if filename is None:
            filename = tkfd.askopenfilename(
                initialdir=container, 
                title='Select file', 
                filetypes=(('json files','*.json'),('all files','*.*')))
        try:
            config = read_config(filename)
        except IOError:
            config = None
        parent.update_config(config)
        
    def save_json(self, adict):
        filename = self.filename.get()
        with open(filename, 'w') as f:
            json.dump(adict, f)
            
class HidePrint():
    def __enter__(self):
        self._orig_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._orig_stdout

class DINGOApp(tk.Tk):
    def __init__(self, filename=None, label='DINGO', master=None):
        if master is None:
            #tk.NoDefaultRoot()
            tk.Tk.__init__(self)
        else:
            tk.Tk.__init__(self, master=master)
        self.grid_columnconfigure(0,weight=1)
        self.grid_rowconfigure(0,weight=1)
        self.resizable(True, False)
        self.create_widgets(configpath=filename)
        self.grid_widgets()
        self.update_options(self.file_manip.config)
        self.mainloop()
#        DINGOApp.main()
    
    @classmethod
    def main(cls):
        tk.NoDefaultRoot()
        root = tk.Tk()
        app = cls(root)
        app.grid_columnconfigure(0,weight=1)
        app.grid_rowconfigure(0,weight=1)
        root.resizable(True, False)
        root.mainloop()
            
    def create_widgets(self, configpath=None):
        self.options = DINGOOptions(self)
        self.file_manip = DINGOFile(self)
        
        if configpath is not None:
            self.aDINGO = DINGO(configpath=configpath)
        if self.aconfig is not None:
            try:
                self.aDINGO.check_input_fields('DINGOApp', self.aconfig, DINGO.default_expected_keys)
            except Exception as err:
                DINGOApp.handle_create_error(err)
                
        self.run_plugin = tk.Entry(self, text='Linear')
        self.run_plugin_args = Extra_entry(self)
        self.run_button = tk.Button(self, text='Run', command=lambda : self.run_DINGO(configpath))
        self.quit_button = tk.Button(self, text='Quit', command=lambda : self.master.destroy)
        
    def grid_widgets(self):
        options = dict(sticky='nsew', padx=4, pady=4)
        self.file_manip.grid(row=0, column=0, **options)
        self.options.grid(row=1, column=0, **options)
        self.run_plugin.grid(row=2, column=0, **options)
        self.run_plugin_args.grid(row=2, column=1, **options)
        self.runButton.grid(row=3, column=0, **options)
        self.quitButton.grid(row=3, column=1, **options)
        
    def update_config(self, config):
        self.aconfig = config
        
        if config is not None:
            pprint(self.__dict__, indent = 2)
            for k,v in config.iteritems():
                print('UpCo: k: {}, v: {}'.format(k,v))
                self.options.make_options_widget(k,v)
            
    @staticmethod        
    def handle_create_error(err):
        tkmb.showerror(title='DINGO Creation Error', message=err['traceback'])
    
    @staticmethod
    def handle_run_error(err):
        raise tkmb.showerror(title='DINGO Run Error', message=err['traceback'])
        
    def run_DINGO_mock(self, configpath):
        print('I did not run DINGO')
        msg='I did not run DINGO'
        raise(Exception(msg))
            
    def run_DINGO(self, config):
        with HidePrint():
            self.aDINGO = DINGO()
        try:
            self.aDINGO.create_wf_from_config(cfg=config)
        except Exception as err:
            DINGOApp.handle_create_error(err)
            
        try:
            self.aDINGO.run(**runoptions)
        except Exception as err:
            DINGOApp.handle_run_error(err)
        

IndentationError: expected an indented block (<ipython-input-1-fcd85528ea89>, line 141)

In [ ]:
aconfigpath='/home/pirc/Desktop/five-phantom-trip/Boston/ProcessDTI.json'
DINGOApp()